In [1]:
import pandas as pd
from rdkit import Chem
# discussion of circular fingerprints: https://pubs.acs.org/doi/10.1021/ci100050t
from rdkit.Chem import AllChem
import numpy as np
from tqdm import tqdm
tqdm.pandas()
import os
import pickle
#Other fingerprint types to explore? 
#useful example: https://medium.com/@gurkamaldeol/predicting-environmental-carcinogens-with-logistic-regression-knn-gradient-boosting-and-7973f88eb8b3

In [2]:
datasets=[i for i in os.listdir('data_cleaned') if i[-4:]=='.csv']
datasets

['clintox.csv',
 'sol_del.csv',
 'HIV.csv',
 'bace.csv',
 'tox21.csv',
 'deepchem_Lipophilicity.csv']

In [8]:
data_map={
    'HIV.csv': {'target':'HIV_active','structure':'smiles'},
    'bace.csv':{'target':'active','structure':'mol'},
    'tox21.csv':{'target':'NR-AhR','structure':'smiles'},
    'clintox.csv':{'target':'CT_TOX','structure':'smiles'},
    'Solubility_delaney-processed.csv':{'target':'binned_sol','structure':'smiles'},
    'deepchem_Lipophilicity.csv':{'target':'drug_like','structure':'smiles'}   
}

In [9]:
data_map[datasets[0]]

{'target': 'CT_TOX', 'structure': 'smiles'}

In [10]:
def generate_fingerprint(smiles,radius,bits):
    mol=Chem.MolFromSmiles(smiles)
    fp=AllChem.GetMorganFingerprintAsBitVect(mol,radius,bits)
    return(np.array(fp))

In [11]:
# test this: 
generate_fingerprint('C=C=C',2,1024)

array([0, 0, 0, ..., 0, 0, 0])

In [12]:
# build a test case with HIV data:
radius=2
bits=1024
df=pd.read_csv('data_cleaned/HIV.csv')
df['fp']=df['smiles'].progress_apply(lambda x: generate_fingerprint(x,radius,bits))
df.head(2)

 85%|██████████████████████████████████████████████████████▌         | 35075/41127 [00:31<00:07, 854.15it/s][15:53:33] WARNING: not removing hydrogen atom without neighbors
[15:53:33] WARNING: not removing hydrogen atom without neighbors
100%|███████████████████████████████████████████████████████████████| 41127/41127 [00:38<00:00, 1082.04it/s]


,Unnamed: 0,smiles,activity,HIV_active,fp
0,0,CCC1=[O+][Cu-3]2([O+]=C(CC)C1)[O+]=C(CC)CC(CC)...,CI,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,1,C(=Cc1ccccc1)C1=[O+][Cu-3]2([O+]=C(C=Cc3ccccc3...,CI,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [13]:
# split the data:
from sklearn.model_selection import train_test_split

X=[generate_fingerprint(mol,2,1024) for mol in tqdm(df['smiles'])]
y=df['HIV_active'].to_list()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

 85%|██████████████████████████████████████████████████████▌         | 35081/41127 [00:35<00:07, 801.38it/s][15:54:16] WARNING: not removing hydrogen atom without neighbors
[15:54:16] WARNING: not removing hydrogen atom without neighbors
100%|████████████████████████████████████████████████████████████████| 41127/41127 [00:42<00:00, 967.54it/s]


In [14]:
#Load the Models:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC


#Load the Metrics:
from sklearn.model_selection import cross_validate
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss

#RandomForestClassifier(max_depth=2, random_state=0)
#KNeighborsClassifier(n_neighbors=5)
#GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=1, random_state=0)
#SVC(C=1.0, kernel='rbf', degree=3, gamma='scale',probability=True)


# Setup Cross validation:
scoring = ['accuracy', 'f1','roc_auc','neg_log_loss']
clf= LogisticRegression(random_state=0,solver='lbfgs',max_iter=1000,verbose=False)
scores = cross_validate(clf, X_train, y_train, scoring=scoring, cv=5,return_train_score=True)

In [15]:
scores

{'fit_time': array([3.04000521, 2.9063077 , 3.0485034 , 3.0406611 , 3.4751327 ]),
 'score_time': array([0.13809896, 0.09592152, 0.10188508, 0.10339856, 0.10300469]),
 'test_accuracy': array([0.96887664, 0.96920084, 0.96839034, 0.96660723, 0.96660723]),
 'train_accuracy': array([0.97442859, 0.97483385, 0.97479332, 0.97487437, 0.9747528 ]),
 'test_f1': array([0.34693878, 0.37086093, 0.37299035, 0.33548387, 0.31333333]),
 'train_f1': array([0.47980214, 0.49056604, 0.48595041, 0.49180328, 0.48976249]),
 'test_roc_auc': array([0.784781  , 0.77769059, 0.79928654, 0.79040174, 0.7819159 ]),
 'train_roc_auc': array([0.92498168, 0.92358746, 0.92086244, 0.9240289 , 0.92612064]),
 'test_neg_log_loss': array([-0.12627385, -0.12951767, -0.12225941, -0.12971206, -0.13112323]),
 'train_neg_log_loss': array([-0.08466068, -0.08375676, -0.08530655, -0.0841092 , -0.0839419 ])}

In [16]:
for score in scores:
    print(score,scores[score].mean(),'+/-',scores[score].std())

fit_time 3.1021220207214357 +/- 0.19390261435995465
score_time 0.10846176147460937 +/- 0.015061801418536861
test_accuracy 0.9679364564759281 +/- 0.0011155552299617674
train_accuracy 0.9747365861565893 +/- 0.0001592387964369198
test_f1 0.3479214521322693 +/- 0.02239047288697682
train_f1 0.487576872571201 +/- 0.0043512371238090255
test_roc_auc 0.7868151541942294 +/- 0.007480604993680964
train_roc_auc 0.923916223603868 +/- 0.001757327127758757
test_neg_log_loss -0.12777724372882976 +/- 0.0031835495035589636
train_neg_log_loss -0.08435502050890412 +/- 0.0005635325103984394


In [17]:
#Load the Models:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC


#Load the Metrics:
from sklearn.model_selection import cross_validate
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss

scoring = ['accuracy', 'f1','roc_auc','neg_log_loss']

models={'Logistic Regression':LogisticRegression(random_state=0,solver='lbfgs',max_iter=1000,verbose=False),
        'Random Forest':RandomForestClassifier(max_depth=2, random_state=0),
        'KNN': KNeighborsClassifier(n_neighbors=5, n_jobs=-1),
        'Gradient Boosted Tree': GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=1, random_state=0),
        'SVM':SVC(C=1.0, kernel='linear', degree=3, gamma='scale',probability=True),
        'Dummy_Most_Frequent':DummyClassifier(strategy="most_frequent")
       }

In [18]:
scores_dict={}

for model in tqdm(models):
    print(f'Training{model}')
    clf=models[model]
    scores_dict.update({model : cross_validate(clf, X_train, y_train, scoring=scoring, cv=5,return_train_score=True)})


  0%|                                                                                 | 0/5 [00:00<?, ?it/s]

TrainingLogistic Regression


 20%|██████████████▌                                                          | 1/5 [00:18<01:13, 18.42s/it]

TrainingRandom Forest


 40%|█████████████████████████████▏                                           | 2/5 [00:32<00:47, 15.70s/it]

...

 60%|███████████████████████████████████████████▏                            | 3/5 [07:44<06:51, 205.74s/it]

TrainingGradient Boosted Tree


 80%|█████████████████████████████████████████████████████████▌              | 4/5 [11:41<03:38, 218.32s/it]

TrainingSVM


100%|█████████████████████████████████████████████████████████████████████| 5/5 [1:40:14<00:00, 1202.91s/it]


In [19]:
scores_dict

{'Logistic Regression': {'fit_time': array([2.87739778, 3.22573662, 2.98598957, 3.13190079, 3.74218917]),
  'score_time': array([0.097754  , 0.10137129, 0.12909412, 0.10858965, 0.12678814]),
  'test_accuracy': array([0.96887664, 0.96920084, 0.96839034, 0.96660723, 0.96660723]),
  'train_accuracy': array([0.97442859, 0.97483385, 0.97479332, 0.97487437, 0.9747528 ]),
  'test_f1': array([0.34693878, 0.37086093, 0.37299035, 0.33548387, 0.31333333]),
  'train_f1': array([0.47980214, 0.49056604, 0.48595041, 0.49180328, 0.48976249]),
  'test_roc_auc': array([0.784781  , 0.77769059, 0.79928654, 0.79040174, 0.7819159 ]),
  'train_roc_auc': array([0.92498168, 0.92358746, 0.92086244, 0.9240289 , 0.92612064]),
  'test_neg_log_loss': array([-0.12627385, -0.12951767, -0.12225941, -0.12971206, -0.13112323]),
  'train_neg_log_loss': array([-0.08466068, -0.08375676, -0.08530655, -0.0841092 , -0.0839419 ])},
 'Random Forest': {'fit_time': array([2.10604024, 2.17551517, 2.13161039, 2.16317821, 2.15692306

In [25]:
for key in scores_dict.keys():
    print(key)
    for score in scores_dict[key]:
        print(score,scores_dict[key][score].mean().round(2),'+/-',scores_dict[key][score].std().round(2))

Logistic Regression
fit_time 3.19 +/- 0.3
score_time 0.11 +/- 0.01
test_accuracy 0.97 +/- 0.0
train_accuracy 0.97 +/- 0.0
test_f1 0.35 +/- 0.02
train_f1 0.49 +/- 0.0
test_roc_auc 0.79 +/- 0.01
train_roc_auc 0.92 +/- 0.0
test_neg_log_loss -0.13 +/- 0.0
train_neg_log_loss -0.08 +/- 0.0
Random Forest
fit_time 2.15 +/- 0.02
score_time 0.13 +/- 0.01
test_accuracy 0.97 +/- 0.0
train_accuracy 0.97 +/- 0.0
test_f1 0.0 +/- 0.0
train_f1 0.0 +/- 0.0
test_roc_auc 0.72 +/- 0.02
train_roc_auc 0.74 +/- 0.0
test_neg_log_loss -0.14 +/- 0.0
train_neg_log_loss -0.14 +/- 0.0
KNN
fit_time 0.07 +/- 0.0
score_time 17.97 +/- 2.24
test_accuracy 0.97 +/- 0.0
train_accuracy 0.98 +/- 0.0
test_f1 0.47 +/- 0.04
train_f1 0.56 +/- 0.01
test_roc_auc 0.78 +/- 0.02
train_roc_auc 0.98 +/- 0.0
test_neg_log_loss -0.53 +/- 0.05
train_neg_log_loss -0.05 +/- 0.0
Gradient Boosted Tree
fit_time 46.95 +/- 0.58
score_time 0.11 +/- 0.01
test_accuracy 0.97 +/- 0.0
train_accuracy 0.97 +/- 0.0
test_f1 0.1 +/- 0.02
train_f1 0.11 +/- 0

In [72]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0,solver='lbfgs',max_iter=1000)
clf.fit(X_train,y_train)
y_train_pred=clf.predict(X_train)
y_test_pred=clf.predict(X_test)



In [74]:
# Score the model
result={}
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

result.update({'train':{'accuracy':accuracy_score(y_train, y_train_pred),
                       'f1':f1_score(y_train, y_train_pred)}})

result.update({'test':{'accuracy':accuracy_score(y_test, y_test_pred),
                       'f1':f1_score(y_test, y_test_pred)}})


In [76]:
print('Logistic Regression Result')
result

Logistic Regression Result


{'train': {'accuracy': 0.9735775652455827, 'f1': 0.45702864756828776},
 'test': {'accuracy': 0.9693639369772418, 'f1': 0.3835616438356164}}

In [77]:
from sklearn.dummy import DummyClassifier
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X_train, y_train)
y_train_pred=dummy_clf.predict(X_train)
y_test_pred=dummy_clf.predict(X_test)

result_dummy={}
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

result_dummy.update({'train':{'accuracy':accuracy_score(y_train, y_train_pred),
                       'f1':f1_score(y_train, y_train_pred)}})

result_dummy.update({'test':{'accuracy':accuracy_score(y_test, y_test_pred),
                       'f1':f1_score(y_test, y_test_pred)}})
print('Dummy Result')
result_dummy

Dummy Result


{'train': {'accuracy': 0.965051061760415, 'f1': 0.0},
 'test': {'accuracy': 0.9645010698307722, 'f1': 0.0}}

In [83]:
df['HIV_active'].value_counts()

0    39684
1     1443
Name: HIV_active, dtype: int64

In [84]:
df['HIV_active'].value_counts()/df.shape[0]

0    0.964914
1    0.035086
Name: HIV_active, dtype: float64

In [ ]:
# pickle the model
with open('model_pkl', 'wb') as files:
    pickle.dump(model, files)